In [1]:
import os

from pyspark import SparkConf
from pyspark.sql import SparkSession

In [2]:
access_key = os.getenv("AWS_ACCESS_KEY_ID")
secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")

s3_url = 's3a://genius-lyrics-dataset/song_lyrics.csv'

In [3]:
conf = SparkConf()
conf.set('spark.hadoop.fs.s3a.access.key',access_key)
conf.set('spark.hadoop.fs.s3a.secret.key', secret_key)
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4')

In [4]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

23/11/16 21:40:35 WARN Utils: Your hostname, Shashanks-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.62 instead (on interface en0)
23/11/16 21:40:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/reddy/opt/anaconda3/envs/AILyricsGen/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/reddy/.ivy2/cache
The jars for the packages stored in: /Users/reddy/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1f441ff8-f765-41c6-a882-8226bbc6b845;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 187ms :: artifacts dl 8ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------

In [5]:
df = spark.read.option("header", "true") \
        .option("multiline", "true") \
            .option("quote", "\"") \
                .option("escape", "\"") \
                .csv(s3_url)

23/11/16 21:40:40 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [6]:
df.show()

+--------------------+---+---------+----+------+--------------------+--------------------+---+-------------+-----------+--------+
|               title|tag|   artist|year| views|            features|              lyrics| id|language_cld3|language_ft|language|
+--------------------+---+---------+----+------+--------------------+--------------------+---+-------------+-----------+--------+
|           Killa Cam|rap|  Cam'ron|2004|173166|{"Cam\\'ron","Ope...|[Chorus: Opera St...|  1|           en|         en|      en|
|          Can I Live|rap|    JAY-Z|1996|468624|                  {}|[Produced by Irv ...|  3|           en|         en|      en|
|   Forgive Me Father|rap| Fabolous|2003|  4743|                  {}|Maybe cause I'm e...|  4|           en|         en|      en|
|        Down and Out|rap|  Cam'ron|2004|144404|{"Cam\\'ron","Kan...|[Produced by Kany...|  5|           en|         en|      en|
|              Fly In|rap|Lil Wayne|2005| 78271|                  {}|[Intro]\nSo they ...|

In [17]:
df.fillna("language_cld3", "language_ft").show()

DataFrame[title: string, tag: string, artist: string, year: string, views: string, features: string, lyrics: string, id: string, language_cld3: string, language_ft: string, language: string]

In [8]:
df.na.fill("language_cld3", "language").show()

+--------------------+---+---------+----+------+--------------------+--------------------+---+-------------+-----------+--------+
|               title|tag|   artist|year| views|            features|              lyrics| id|language_cld3|language_ft|language|
+--------------------+---+---------+----+------+--------------------+--------------------+---+-------------+-----------+--------+
|           Killa Cam|rap|  Cam'ron|2004|173166|{"Cam\\'ron","Ope...|[Chorus: Opera St...|  1|           en|         en|      en|
|          Can I Live|rap|    JAY-Z|1996|468624|                  {}|[Produced by Irv ...|  3|           en|         en|      en|
|   Forgive Me Father|rap| Fabolous|2003|  4743|                  {}|Maybe cause I'm e...|  4|           en|         en|      en|
|        Down and Out|rap|  Cam'ron|2004|144404|{"Cam\\'ron","Kan...|[Produced by Kany...|  5|           en|         en|      en|
|              Fly In|rap|Lil Wayne|2005| 78271|                  {}|[Intro]\nSo they ...|

In [9]:
df.na.drop()

DataFrame[title: string, tag: string, artist: string, year: string, views: string, features: string, lyrics: string, id: string, language_cld3: string, language_ft: string, language: string]

In [15]:
df_1 = df.drop("language_ft", "language")

In [16]:
df_1.show()

+--------------------+---+---------+----+------+--------------------+--------------------+---+-------------+
|               title|tag|   artist|year| views|            features|              lyrics| id|language_cld3|
+--------------------+---+---------+----+------+--------------------+--------------------+---+-------------+
|           Killa Cam|rap|  Cam'ron|2004|173166|{"Cam\\'ron","Ope...|[Chorus: Opera St...|  1|           en|
|          Can I Live|rap|    JAY-Z|1996|468624|                  {}|[Produced by Irv ...|  3|           en|
|   Forgive Me Father|rap| Fabolous|2003|  4743|                  {}|Maybe cause I'm e...|  4|           en|
|        Down and Out|rap|  Cam'ron|2004|144404|{"Cam\\'ron","Kan...|[Produced by Kany...|  5|           en|
|              Fly In|rap|Lil Wayne|2005| 78271|                  {}|[Intro]\nSo they ...|  6|           en|
|      Lollipop Remix|rap|Lil Wayne|2008|580832|{"Kanye West","St...|[Intro: Lil Wayne...|  7|           en|
|          Im Not Y

In [19]:
spark.stop()